In [3]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
# Create SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('test-download') \
                    .getOrCreate()

print(f'The PySpark {spark.version} version is running...')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/22 08:50:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


The PySpark 3.5.0 version is running...


### download the file

In [8]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2024-02-22 08:51:01--  http://wget/
Resolving wget (wget)... failed: nodename nor servname provided, or not known.
wget: unable to resolve host address ‘wget’
--2024-02-22 08:51:01--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.117.240, 52.217.115.0, 52.217.166.168, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.117.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv.1’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2024-02-22 08:51:01 (68.3 MB/s) - ‘taxi+_zone_lookup.csv.1’ saved [12322/12322]

FINISHED --2024-02-22 08:51:01--
Total wall clock time: 0.3s
Downloaded: 1 files, 12K in 0s (68.3 MB/s)


In [10]:
!head taxi+_zone_lookup.csv

"LocationID","Borough","Zone","service_zone"
1,"EWR","Newark Airport","EWR"
2,"Queens","Jamaica Bay","Boro Zone"
3,"Bronx","Allerton/Pelham Gardens","Boro Zone"
4,"Manhattan","Alphabet City","Yellow Zone"
5,"Staten Island","Arden Heights","Boro Zone"
6,"Staten Island","Arrochar/Fort Wadsworth","Boro Zone"
7,"Queens","Astoria","Boro Zone"
8,"Queens","Astoria Park","Boro Zone"
9,"Queens","Auburndale","Boro Zone"


### read file into dataframe

In [13]:
df = spark.read\
     .option("header","true")\
     .csv('taxi+_zone_lookup.csv')
df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

### convert the file to parquet and save it as zones

In [18]:
df.write.parquet('zones')